In [ ]:
# !vi ~/.keras/keras.json

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
from tensorflow.python.client import device_lib
import tensorflow as tf

print("GPU Available: ", tf.test.is_gpu_available())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print('GPU is available')

In [1]:
%cd /root/mobilenet_v2_ssdlite_keras

/root/mobilenet_v2_ssdlite_keras


In [2]:
import sys
sys.path.append('/root/mobilenet_v2_ssdlite_keras')

In [3]:
from __future__ import division
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from losses.keras_ssd_loss import SSDLoss
from utils.object_detection_2d_data_generator import DataGenerator
from utils.object_detection_2d_geometric_ops import Resize
from utils.object_detection_2d_photometric_ops import ConvertTo3Channels
from utils.data_augmentation_chain_original_ssd import SSDDataAugmentation
from utils.coco import get_coco_category_maps
from utils.ssd_input_encoder import SSDInputEncoder
# from models.keras_mobilenet_v2_ssdlite import mobilenet_v2_ssd
from layers.AnchorBoxesLayer import AnchorBoxes
from layers.DecodeDetectionsLayer import DecodeDetections
from layers.DecodeDetectionsFastLayer import DecodeDetectionsFast
from models.graphs.mobilenet_v2_ssdlite_praph import mobilenet_v2_ssdlite
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K 
from tensorflow.keras.layers import Input, Lambda, Activation, Conv2D, \
    DepthwiseConv2D, Reshape, Concatenate, BatchNormalization, ReLU
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from glob import glob
from tensorflow import keras

2021-12-07 01:07:20.127052: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# tf.debugging.set_log_device_placement(True)

# OID txt TO XML

## 1.conver_annotation

In [ ]:
%cd /root/OIDv4_ToolKit/

In [ ]:
len(glob('/root/OIDv4_ToolKit/OID/Dataset/train/Hat/*.jpg'))

In [ ]:
len(glob('/root/OIDv4_ToolKit/OID/Dataset/train/Helmet/*.jpg'))

In [ ]:
!python /root/OIDv4_ToolKit/convert_annotations.py

## 2. txt to xml

In [ ]:
pwd

In [ ]:
%cd /root/OIDv4_ToolKit/

In [ ]:
ls

In [ ]:
!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/train/Hat\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/train_annotation/


In [ ]:
from glob import glob

In [ ]:
# xml 파일 갯수
print(len(glob('/root/OIDv4_ToolKit/OID/Dataset/train_annotation/*.xml')))
# txt 파일 갯수
print(len(glob('/root/OIDv4_ToolKit/OID/Dataset/train/Hat/Label/*.txt')))

In [ ]:
!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/train/Helmet\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/train_annotation/


In [ ]:
# xml 파일 갯수
print(len(glob('/root/OIDv4_ToolKit/OID/Dataset/train_annotation/*.xml'))\
     -len(glob('/root/OIDv4_ToolKit/OID/Dataset/train/Helmet/Label/*.txt')))

# txt 파일 갯수
print(len(glob('/root/OIDv4_ToolKit/OID/Dataset/train/Helmet/Label/*.txt')))

In [ ]:
!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/train/Bicycle_helmet\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/train_annotation/


In [ ]:
!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/train/Human_head\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/train_annotation/


In [ ]:
print(len(glob('/root/OIDv4_ToolKit/OID/Dataset/train_annotation/*.xml')))

In [ ]:
!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/test/Hat\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/test_annotation/

!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/test/Helmet\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/test_annotation/

!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/test/Bicycle_helmet\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/test_annotation/

!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/test/Human_head\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/test_annotation/


In [ ]:
!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/validation/Hat\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/validation_annotation/

!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/validation/Helmet\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/validation_annotation/

!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/validation/Bicycle_helmet\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/validation_annotation/

!python3 OIDv4_to_VOC.py\
--sourcepath /root/OIDv4_ToolKit/OID/Dataset/validation/Human_head\
--dest_path /root/OIDv4_ToolKit/OID/Dataset/validation_annotation/


## 이미지 폴더 통합

In [ ]:
from tqdm import tqdm
import os
for i in tqdm(['train', 'test', 'validation']):
    for j in ['Bicycle_helmet', 'Hat', 'Helmet', 'Human_head']:
        for k in glob(f'/root/OIDv4_ToolKit/OID/Dataset/{i}/{j}/*.txt'):
            os.remove(k)

In [ ]:
from tqdm import tqdm
import shutil
for i in tqdm(['train', 'test', 'validation']):
    for j in ['Bicycle_helmet', 'Hat', 'Helmet', 'Human_head']:
        for k in glob(f'/root/OIDv4_ToolKit/OID/Dataset/{i}/{j}/*.jpg'):
            shutil.copyfile(k, f'/root/OIDv4_ToolKit/OID/Dataset/{i}_all/{k.split("/")[-1]}')

In [ ]:
for i in tqdm(['train', 'test', 'validation']):
    with open(f'/root/OIDv4_ToolKit/OID/Dataset/{i}.txt', 'w') as f: 
        for k in glob(f'/root/OIDv4_ToolKit/OID/Dataset/{i}_all/*.jpg'):
            f.write(k.split('/')[-1].split('.')[0] + '\n')

# model_config

In [4]:
K.clear_session()

In [5]:
# model config
batch_size = 1024
image_size = (300, 300, 3)
n_classes = 4
mode = 'training'
l2_regularization = 0.0005
min_scale = 0.1
max_scale = 0.9
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05]
scales = scales_pascal
aspect_ratios_global = None
aspect_ratios  = [[1.0, 2.0, 0.5], 
                [1.0, 2.0, 0.5, 3.0, 1.0 / 3.0], 
                [1.0, 2.0, 0.5, 3.0, 1.0 / 3.0],
                [1.0, 2.0, 0.5, 3.0, 1.0 / 3.0], 
                [1.0, 2.0, 0.5], 
                [1.0, 2.0, 0.5]]
two_boxes_for_ar1 = True
steps = None
offsets = None
clip_boxes = False
variances = [0.1, 0.1, 0.2, 0.2]

### learning_rate

In [ ]:
# # learning rate
# def lr_schedule(epoch):
#     if epoch < 200:
#         return 0.0001
#     elif epoch < 500:
#         return 0.00005
#     else:
#         return 0.00001

### 훈련 가능한 레이블 설정

In [6]:
# set trainable layers
def set_trainable(layer_regex, keras_model=None, indent=0, verbose=1):
    # In multi-GPU training, we wrap the model. Get layers
    # of the inner model because they have the weights.
    layers = keras_model.inner_model.layers if hasattr(keras_model, "inner_model") \
        else keras_model.layers

    for layer in layers:
        # Is the layer a model?
        if layer.__class__.__name__ == 'Model':
            print("In model: ", layer.name)
            set_trainable(
                layer_regex, keras_model=layer)
            continue

        if not layer.weights:
            continue
        # Is it trainable?
        trainable = bool(re.fullmatch(layer_regex, layer.name))
        # Update layer. If layer is a container, update inner layer.
        if layer.__class__.__name__ == 'TimeDistributed':
            layer.layer.trainable = trainable
        else:
            layer.trainable = trainable
        # Print trainable layer names
        if trainable and verbose > 0:
            print("{}{:20}   ({})".format(" " * indent, layer.name, layer.__class__.__name__))


### predict_block

In [7]:
def predict_block(inputs, out_channel, sym, id):
    name = 'ssd_' + sym + '{}'.format(id)
    x = DepthwiseConv2D(kernel_size=3, strides=1,
                           activation=None, use_bias=False, padding='same', name=name + '_dw_conv')(inputs)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999, name=name + '_dw_bn')(x)
    x = ReLU(6., name=name + '_dw_relu')(x)

    x = Conv2D(out_channel, kernel_size=1, padding='same', use_bias=False,
                  activation=None, name=name + 'conv2')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999, name=name + 'conv2_bn')(x)
    return x


### model

In [8]:
def mobilenet_v2_ssd(image_size,
            n_classes,
            mode='training',
            l2_regularization=0.0005,
            min_scale=None,
            max_scale=None,
            scales=None,
            aspect_ratios_global=None,
            aspect_ratios_per_layer=aspect_ratios,
            two_boxes_for_ar1=True,
            steps=[8, 16, 32, 64, 100, 300],
            offsets=None,
            clip_boxes=False,
            variances=[0.1, 0.1, 0.2, 0.2],
            coords='centroids',
            normalize_coords=True,
            subtract_mean=[123, 117, 104],
            divide_by_stddev=None,
            swap_channels=[2, 1, 0],
            confidence_thresh=0.01,
            iou_threshold=0.45,
            top_k=100,
            nms_max_output_size=400,
            return_predictor_sizes=False):
   

    n_predictor_layers = 6  # The number of predictor conv layers in the network is 6 for the original SSD300.
    n_classes += 1  # Account for the background class.
    l2_reg = l2_regularization  # Make the internal name shorter.
    img_height, img_width, img_channels = image_size[0], image_size[1], image_size[2]

    ############################################################################
    # Get a few exceptions out of the way.
    ############################################################################

    if aspect_ratios_global is None and aspect_ratios_per_layer is None:
        raise ValueError(
            "`aspect_ratios_global` and `aspect_ratios_per_layer` \
            cannot both be None. At least one needs to be specified.")
    if aspect_ratios_per_layer:
        if len(aspect_ratios_per_layer) != n_predictor_layers:
            raise ValueError(
                "It must be either aspect_ratios_per_layer is None or len(aspect_ratios_per_layer) == {}, \
                but len(aspect_ratios_per_layer) == {}.".format(
                    n_predictor_layers, len(aspect_ratios_per_layer)))

    if (min_scale is None or max_scale is None) and scales is None:
        raise ValueError("Either `min_scale` and `max_scale` or `scales` need to be specified.")
    if scales:
        if len(scales) != n_predictor_layers + 1:
            raise ValueError("It must be either scales is None or len(scales) == {}, but len(scales) == {}.".format(
                n_predictor_layers + 1, len(scales)))
    # If no explicit list of scaling factors was passed,
    # compute the list of scaling factors from `min_scale` and `max_scale`
    else:
        scales = np.linspace(min_scale, max_scale, n_predictor_layers + 1)

    if len(variances) != 4:
        raise ValueError("4 variance values must be pased, but {} values were received.".format(len(variances)))
    variances = np.array(variances)
    if np.any(variances <= 0):
        raise ValueError("All variances must be >0, but the variances given are {}".format(variances))

    if (not (steps is None)) and (len(steps) != n_predictor_layers):
        raise ValueError("You must provide at least one step value per predictor layer.")

    if (not (offsets is None)) and (len(offsets) != n_predictor_layers):
        raise ValueError("You must provide at least one offset value per predictor layer.")

    ############################################################################
    # Compute the anchor box parameters.
    ############################################################################

    # Set the aspect ratios for each predictor layer. These are only needed for the anchor box layers.
    if aspect_ratios_per_layer:
        aspect_ratios = aspect_ratios_per_layer
    else:
        aspect_ratios = [aspect_ratios_global] * n_predictor_layers
    print("여기")
    # Compute the number of boxes to be predicted per cell for each predictor layer.
    # We need this so that we know how many channels the predictor layers need to have.
    if aspect_ratios_per_layer:
        n_boxes = []
        for ar in aspect_ratios_per_layer:
            if (1 in ar) & two_boxes_for_ar1:
                n_boxes.append(len(ar) + 1)  # +1 for the second box for aspect ratio 1
            else:
                n_boxes.append(len(ar))
    # If only a global aspect ratio list was passed,
    # then the number of boxes is the same for each predictor layer
    else:
        if (1 in aspect_ratios_global) & two_boxes_for_ar1:
            n_boxes = len(aspect_ratios_global) + 1
        else:
            n_boxes = len(aspect_ratios_global)
        n_boxes = [n_boxes] * n_predictor_layers

    if steps is None:
        steps = [None] * n_predictor_layers
    if offsets is None:
        offsets = [None] * n_predictor_layers

    ############################################################################
    # Define functions for the Lambda layers below.
    ############################################################################

    def identity_layer(tensor):
        return tensor

    def input_mean_normalization(tensor):
        return tensor - np.array(subtract_mean)

    def input_stddev_normalization(tensor):
        return tensor / np.array(divide_by_stddev)

    def input_channel_swap(tensor):
        if len(swap_channels) == 3:
            return K.stack(
                [tensor[..., swap_channels[0]], tensor[..., swap_channels[1]], tensor[..., swap_channels[2]]], axis=-1)
        elif len(swap_channels) == 4:
            return K.stack([tensor[..., swap_channels[0]], tensor[..., swap_channels[1]], tensor[..., swap_channels[2]],
                            tensor[..., swap_channels[3]]], axis=-1)

    ############################################################################
    # Build the network.
    ############################################################################

    x = Input(shape=(img_height, img_width, img_channels))

    # The following identity layer is only needed so that the subsequent lambda layers can be optional.
    x1 = Lambda(identity_layer, output_shape=(img_height, img_width, img_channels), name='identity_layer')(x)

    tmp_shape = K.int_shape(x1)

    if not (subtract_mean is None):
        x1 = Lambda(input_mean_normalization, output_shape=(img_height, img_width, img_channels),
                    name='input_mean_normalization')(x1)
    if not (divide_by_stddev is None):
        x1 = Lambda(input_stddev_normalization, output_shape=(img_height, img_width, img_channels),
                    name='input_stddev_normalization')(x1)
    if swap_channels:
        x1 = Lambda(
            input_channel_swap, output_shape=(img_height, img_width, img_channels), name='input_channel_swap')(x1)

    links = mobilenet_v2_ssdlite(x1)

    link1_cls = predict_block(links[0], n_boxes[0] * n_classes, 'cls', 1)
    link2_cls = predict_block(links[1], n_boxes[1] * n_classes, 'cls', 2)
    link3_cls = predict_block(links[2], n_boxes[2] * n_classes, 'cls', 3)
    link4_cls = predict_block(links[3], n_boxes[3] * n_classes, 'cls', 4)
    link5_cls = predict_block(links[4], n_boxes[4] * n_classes, 'cls', 5)
    link6_cls = predict_block(links[5], n_boxes[5] * n_classes, 'cls', 6)

    link1_box = predict_block(links[0], n_boxes[0] * 4, 'box', 1)
    link2_box = predict_block(links[1], n_boxes[1] * 4, 'box', 2)
    link3_box = predict_block(links[2], n_boxes[2] * 4, 'box', 3)
    link4_box = predict_block(links[3], n_boxes[3] * 4, 'box', 4)
    link5_box = predict_block(links[4], n_boxes[4] * 4, 'box', 5)
    link6_box = predict_block(links[5], n_boxes[5] * 4, 'box', 6)
    print("여기2")
    priorbox1 = AnchorBoxes(img_height, img_width, this_scale=scales[0], next_scale=scales[1],
                                             aspect_ratios=aspect_ratios[0],
                                             two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[0],
                                             this_offsets=offsets[0], clip_boxes=clip_boxes,
                                             variances=variances, coords=coords, normalize_coords=normalize_coords,
                                             name='ssd_priorbox_1')(link1_box)
    priorbox2 = AnchorBoxes(img_height, img_width, this_scale=scales[1], next_scale=scales[2],
                                    aspect_ratios=aspect_ratios[1],
                                    two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[1], this_offsets=offsets[1],
                                    clip_boxes=clip_boxes,
                                    variances=variances, coords=coords, normalize_coords=normalize_coords,
                                    name='ssd_priorbox_2')(link2_box)
    priorbox3 = AnchorBoxes(img_height, img_width, this_scale=scales[2], next_scale=scales[3],
                                        aspect_ratios=aspect_ratios[2],
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[2],
                                        this_offsets=offsets[2], clip_boxes=clip_boxes,
                                        variances=variances, coords=coords, normalize_coords=normalize_coords,
                                        name='ssd_priorbox_3')(link3_box)
    priorbox4 = AnchorBoxes(img_height, img_width, this_scale=scales[3], next_scale=scales[4],
                                        aspect_ratios=aspect_ratios[3],
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[3],
                                        this_offsets=offsets[3], clip_boxes=clip_boxes,
                                        variances=variances, coords=coords, normalize_coords=normalize_coords,
                                        name='ssd_priorbox_4')(link4_box)
    priorbox5 = AnchorBoxes(img_height, img_width, this_scale=scales[4], next_scale=scales[5],
                                        aspect_ratios=aspect_ratios[4],
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[4],
                                        this_offsets=offsets[4], clip_boxes=clip_boxes,
                                        variances=variances, coords=coords, normalize_coords=normalize_coords,
                                        name='ssd_priorbox_5')(link5_box)
    priorbox6 = AnchorBoxes(img_height, img_width, this_scale=scales[5], next_scale=scales[6],
                                        aspect_ratios=aspect_ratios[5],
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[5],
                                        this_offsets=offsets[5], clip_boxes=clip_boxes,
                                        variances=variances, coords=coords, normalize_coords=normalize_coords,
                                        name='ssd_priorbox_6')(link6_box)

    # Reshape
    cls1_reshape = Reshape((-1, n_classes), name='ssd_cls1_reshape')(link1_cls)
    cls2_reshape = Reshape((-1, n_classes), name='ssd_cls2_reshape')(link2_cls)
    cls3_reshape = Reshape((-1, n_classes), name='ssd_cls3_reshape')(link3_cls)
    cls4_reshape = Reshape((-1, n_classes), name='ssd_cls4_reshape')(link4_cls)
    cls5_reshape = Reshape((-1, n_classes), name='ssd_cls5_reshape')(link5_cls)
    cls6_reshape = Reshape((-1, n_classes), name='ssd_cls6_reshape')(link6_cls)

    box1_reshape = Reshape((-1, 4), name='ssd_box1_reshape')(link1_box)
    box2_reshape = Reshape((-1, 4), name='ssd_box2_reshape')(link2_box)
    box3_reshape = Reshape((-1, 4), name='ssd_box3_reshape')(link3_box)
    box4_reshape = Reshape((-1, 4), name='ssd_box4_reshape')(link4_box)
    box5_reshape = Reshape((-1, 4), name='ssd_box5_reshape')(link5_box)
    box6_reshape = Reshape((-1, 4), name='ssd_box6_reshape')(link6_box)

    priorbox1_reshape = Reshape((-1, 8), name='ssd_priorbox1_reshape')(priorbox1)
    priorbox2_reshape = Reshape((-1, 8), name='ssd_priorbox2_reshape')(priorbox2)
    priorbox3_reshape = Reshape((-1, 8), name='ssd_priorbox3_reshape')(priorbox3)
    priorbox4_reshape = Reshape((-1, 8), name='ssd_priorbox4_reshape')(priorbox4)
    priorbox5_reshape = Reshape((-1, 8), name='ssd_priorbox5_reshape')(priorbox5)
    priorbox6_reshape = Reshape((-1, 8), name='ssd_priorbox6_reshape')(priorbox6)
    print("여기3")

    cls = Concatenate(axis=1, name='ssd_cls')(
        [cls1_reshape, cls2_reshape, cls3_reshape, cls4_reshape, cls5_reshape, cls6_reshape])

    box = Concatenate(axis=1, name='ssd_box')(
        [box1_reshape, box2_reshape, box3_reshape, box4_reshape, box5_reshape, box6_reshape])

    priorbox = Concatenate(axis=1, name='ssd_priorbox')(
        [priorbox1_reshape, priorbox2_reshape, priorbox3_reshape,
         priorbox4_reshape, priorbox5_reshape, priorbox6_reshape])
    
    print(priorbox.shape)
    cls = Activation('softmax', name='ssd_mbox_conf_softmax')(cls)
    print("여기4")
    predictions = Concatenate(axis=2, name='ssd_predictions')([cls, box, priorbox])

    if mode == 'training':
        model = Model(inputs=x, outputs=predictions)
    elif mode == 'inference':
        decoded_predictions = DecodeDetections(confidence_thresh=confidence_thresh,
                                               iou_threshold=iou_threshold,
                                               top_k=top_k,
                                               nms_max_output_size=nms_max_output_size,
                                               coords=coords,
                                               normalize_coords=normalize_coords,
                                               img_height=img_height,
                                               img_width=img_width,
                                               name='ssd_decoded_predictions')(predictions)
        model = Model(inputs=x, outputs=decoded_predictions)
    elif mode == 'inference_fast':
        decoded_predictions = DecodeDetectionsFast(confidence_thresh=confidence_thresh,
                                                   iou_threshold=iou_threshold,
                                                   top_k=top_k,
                                                   nms_max_output_size=nms_max_output_size,
                                                   coords=coords,
                                                   normalize_coords=normalize_coords,
                                                   img_height=img_height,
                                                   img_width=img_width,
                                                   name='ssd_decoded_predictions')(predictions)
        model = Model(inputs=x, outputs=decoded_predictions)
    else:
        raise ValueError(
            "`mode` must be one of 'training', 'inference' or 'inference_fast', but received '{}'.".format(mode))

    if return_predictor_sizes:
        predictor_sizes = np.array(
            [cls[0]._keras_shape[1:3], cls[1]._keras_shape[1:3], cls[2]._keras_shape[1:3],
             cls[3]._keras_shape[1:3], cls[4]._keras_shape[1:3], cls[5]._keras_shape[1:3]])
        print("return_predictor_sizes")
        return model, predictor_sizes
    else:
        print("모델 리턴")
        return model

### 모델 빌드

In [9]:
# build model
model = mobilenet_v2_ssd(image_size, 
                         n_classes, 
                         mode, 
                         l2_regularization,
                         min_scale, 
                         max_scale, 
                         scales,
                         aspect_ratios_global, 
                         aspect_ratios,
                        # aspect_ratios_per_layer, 
                         two_boxes_for_ar1, 
                         steps,
                         offsets, 
                         clip_boxes, 
                         variances, 
                        #  coords, 
                        #  normalize_coords, 
                        #  subtract_mean,
                        #  divide_by_stddev, 
                        #  swap_channels, 
                        #  confidence_thresh, 
                        #  iou_threshold, 
                        #  top_k,
                        #  nms_max_output_size, 
                        #  return_predictor_sizes
                         )

여기


2021-12-07 01:07:51.535264: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-07 01:07:51.537396: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-07 01:07:52.057099: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 01:07:52.057956: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2021-12-07 01:07:52.057999: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-12-07 01:07:52.062095: I tensorflow/stream_executor/platform/defau

여기2
여기3
(None, 2268, 8)
여기4
모델 리턴


In [ ]:
import gc
gc.collect()

### 옵티마이저, 손실 함수

In [10]:
adam = Adam(learning_rate=0.001)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

### 모델 컴파일

In [11]:
model.compile(optimizer=adam, loss=ssd_loss.compute_loss, metrics = ['acc'])

### 재학습시 모델 로드 

In [ ]:
path = "/root/mobilenet_v2_ssdlite_keras/checkpoint.h5"

model =load_model(
   path, custom_objects={"AnchorBoxes":AnchorBoxes},  compile = False)


adam = Adam(learning_rate=0.01)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
model.compile(optimizer=adam, loss=ssd_loss.compute_loss, metrics=['acc'])

### 데이터 준비 및 로드

In [12]:
# 데이터 로드
train_dataset = DataGenerator()
val_dataset = DataGenerator()

In [13]:
train_images_dir = '/root/OIDv4_ToolKit/OID/Dataset/train_all/'
train_annotations_dir = '/root/OIDv4_ToolKit/OID/Dataset/train_annotation/'
train_image_set_filename = '/root/OIDv4_ToolKit/OID/Dataset/train.txt'


test_images_dir = '/root/OIDv4_ToolKit/OID/Dataset/test_all/'
test_annotations_dir = '/root/OIDv4_ToolKit/OID/Dataset/test_annotation/'
test_image_set_filename     = '/root/OIDv4_ToolKit/OID/Dataset/test.txt'


validation_images_dir = '/root/OIDv4_ToolKit/OID/Dataset/validation_all/'
validation_annotations_dir = '/root/OIDv4_ToolKit/OID/Dataset/validation_annotation/'
validation_image_set_filename     = '/root/OIDv4_ToolKit/OID/Dataset/validation.txt'

In [14]:
classes=['Hat', 'Helmet', 'Bicycle helmet', 'Human head']
# classes=['Hat']

In [ ]:
import os
len(os.listdir('/root/OIDv4_ToolKit/OID/Dataset/train_annotation'))

In [ ]:
len(os.listdir('/root/OIDv4_ToolKit/OID/Dataset/train_all'))

In [15]:
train_dataset.parse_xml(
                        images_dirs = [train_images_dir],
                        image_set_filenames = [train_image_set_filename],
                        annotations_dirs = [train_annotations_dir],
                        classes = classes,
                        include_classes = 'all',
                        exclude_truncated = False,
                        exclude_difficult = False,
                        ret = False
                        )


val_dataset.parse_xml(images_dirs = [validation_images_dir],
                      image_set_filenames = [validation_image_set_filename],
                      annotations_dirs = [validation_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=False,#used to be True
                      ret=False)

Processing image set 'validation.txt': 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 3530/3530 [00:06<00:00, 547.22it/s]


In [ ]:
# train_dataset.create_hdf5_dataset(file_path='/root/mobilenet_v2_ssdlite_keras/train_dataset.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='/root/mobilenet_v2_ssdlite_keras/validation_dataset.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

In [16]:
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 #

In [17]:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width)

In [18]:
# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
#用getlayer来获取输出层的尺寸
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('ssd_cls1conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls2conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls3conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls4conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls5conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls6conv2_bn').output_shape[1:3]]
#encoder把ground truth labels
ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=True)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],# used to be augumentation
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	 38393
Number of images in the validation dataset:	  3530


In [ ]:
# a,b = next(train_generator)
# print(a.shape)
# a = np.expand_dims(a, axis=1)
# print(a.shape)

In [ ]:
pwd

In [ ]:
# 텐서보드
tb_hist = keras.callbacks.TensorBoard(log_dir='./graph',
                                      histogram_freq=0, 
                                      write_graph=True, 
                                      write_images=True)

In [ ]:
K.clear_session()

In [19]:
initial_epoch   = 0
final_epoch     = 1000
# steps_per_epoch = 1 #1035#6000
steps_per_epoch = len(os.listdir('/root/OIDv4_ToolKit/OID/Dataset/train_all')) // batch_size
val_dataset_size = len(os.listdir('/root/OIDv4_ToolKit/OID/Dataset/validation_all'))

# Define model callbacks.
# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='checkpoint.h5',
                                   monitor='val_loss',#used to be val_loss
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   save_freq=int(batch_size * steps_per_epoch))
# int(batch_size * steps_per_epoch)
                                  
#model_checkpoint.best = 

csv_logger = CSVLogger(filename='MobileNetv2_ssdLite_training_log.csv',
                       separator=',',
                       append=True)

# learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
#                                                 verbose=1)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             # learning_rate_scheduler,
             terminate_on_nan]
#              ,tb_hist]

In [20]:
# with tf.device('/device:GPU:0'):
with tf.device('/GPU:0'):
    history = model.fit(train_generator,
                        steps_per_epoch = steps_per_epoch,
                        epochs = final_epoch,
                        callbacks = callbacks,
                        validation_data = val_generator,
                        validation_steps = val_dataset_size//batch_size,
                        initial_epoch = initial_epoch,
                        verbose = 1)

2021-12-07 01:11:31.906067: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-07 01:11:31.906760: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999995000 Hz


Epoch 1/1000
Tensor("Neg:0", shape=(None, 2268), dtype=float32)


2021-12-07 01:11:41.572345: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-12-07 01:11:44.972290: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-12-07 01:11:45.771636: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-12-07 01:11:55.773768: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.75GiB (rounded to 2949120000)requested by op model/bbn_stage1_block2_dw_bn/FusedBatchNormV3
Current allocation summary follows.
2021-12-07 01:11:55.773849: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2021-12-07 01:11:55.773865: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 335, Chunks in use: 335. 83.8KiB allocated for chunks. 83.8KiB in use in

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[1024,32,150,150] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/bbn_stage1_block2_dw_bn/FusedBatchNormV3 (defined at tmp/ipykernel_27381/3988123256.py:10) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[compute_loss/PartitionedCall/Equal/_34]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[1024,32,150,150] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/bbn_stage1_block2_dw_bn/FusedBatchNormV3 (defined at tmp/ipykernel_27381/3988123256.py:10) ]]
Hint: If you want to see a list of allocated t

In [ ]:
# tf.keras.models.save_model(model, '/root/mobilenet_v2_ssdlite_keras/epoch1500_model.h5')

In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# out of memory

import tensorflow as tf

with tf.Graph().as_default():
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)